In [39]:
import pandas as pd
import numpy as np
import pickle as pkl
path = 'C:/Users/alber/OneDrive/Documenti/universita/DATA SCIENCE/Biological Data/PROJECT/GO/'

with open('go.obo','r') as o:
    go_obo = o.read()

with open(path+'GO_human_dataset','r') as o:
    human_GO_ds = pd.read_csv(o, index_col=0, dtype = str)

with open(path+'Fisher_Test/HMM_original_ds_fresults.txt','r') as o:
    original_fresults = pd.read_csv(o, index_col = 0, dtype = str)

with open(path+'Fisher_Test/Architecture_Fisher_tests.pkl','rb') as o:
    arch_datasets = pkl.load(o)

with open(path+'Fisher_Test/PDB_ds_fresults.txt','r') as o:
    pdb_fresults = pd.read_csv(o, index_col=0, dtype = str)    
    
with open(path+'Fisher_Test/New_string_ds_fresults.txt','r') as o:
    string_fresults = pd.read_csv(o, index_col=0, dtype = str)
    


In [50]:
def get_human_tree(go_obo):
    ancestors = {}

    for line in go_obo.split('\n'):
        if line[:2] == 'id':
            go_id = line[7:]
            ancestors[go_id] = []
        if line[:4] == 'is_a':
            ancestors[go_id].append(line[9:16])
    return ancestors

def get_useful_GO(df, thresh):
    df = df.astype({"p-value": float, 'Odds_ratio': float})
    return df[df['p-value'] < thresh]['GO_id'].values

def get_ancestor(GO_id, tree):
    if GO_id in tree:
        return tree[GO_id]
    else:
        return None
    
def create_a_tree(GO_ids, human, level):
    tree = {}

    for GO_id in GO_ids:
        
        ancestors = []
        p = get_ancestor(GO_id,human)
        if p:
            parents = set(p)
            ancestors.append(list(parents))
            while parents and len(ancestors) < level:
                p = [get_ancestor(i,human) for i in parents]
                parents = set([i for c in p for i in c])
                ancestors.append(list(parents))
                
        tree[GO_id] = ancestors
    return tree

In [51]:
# 1 level human GO tree
human_tree = get_human_tree(go_obo)

In [52]:
# retrieve all GO_ids for each dataset
arch_GO_ids1 = arch_datasets['PF00595;']['GO_id'].values
arch_GO_ids2 = arch_datasets['PF00595;PF17820;']['GO_id'].values
original_GO_ids = original_fresults['GO_id'].values
pdb_GO_ids = pdb_fresults['GO_id'].values
string_GO_ids = string_fresults['GO_id'].values

In [53]:
# retrieve all GO_ids having p-value less than 0.05
arch1_useful_GO = list(set(get_useful_GO(arch_datasets['PF00595;'],0.2)))
arch2_useful_GO = list(set(get_useful_GO(arch_datasets['PF00595;PF17820;'],0.2)))
original_useful_GO = list(set(get_useful_GO(original_fresults,0.05)))
string_useful_GO = list(set(get_useful_GO(string_fresults,0.05)))
pdb_useful_GO = list(set(get_useful_GO(pdb_fresults,0.05)))

In [55]:
# retrieve hierarchical trees
original_GO_tree = create_a_tree(original_GO_ids, human_tree,5)
pdb_GO_tree = create_a_tree(pdb_GO_ids, human_tree,5)
string_GO_tree = create_a_tree(string_GO_ids, human_tree,5)
arch_GO_tree1 = create_a_tree(arch_GO_ids1, human_tree,5)
arch_GO_tree2 = create_a_tree(arch_GO_ids2, human_tree,5)

#### IMPROVING GO DATAFRAMES

In [62]:
for key in original_GO_tree.keys():
    if original_GO_tree[key]:
        if original_GO_tree[key][-1]:
            print(original_GO_tree[key][-1])
        else:
            print(original_GO_tree[key][-2])

['0005575']
['0009987', '0050896']
['0005575']
['0003674']
['0008150']
['0051240', '0060284', '0050789', '0050794', '0048518', '0051128', '0045597', '0051094', '0051960']
['0003674']
['0005575']
['0065007', '0050789']
['0008150']
['0051172', '0051174', '0051246', '0031324', '0031323', '0009892', '0010605', '0048523']
['0005488']
['0008150']
['0008150']
['0008150']
['0008150']
['0005575']
['0009987', '0071840', '0016043']
['0065009', '0031325', '0009893', '0051174', '0048522', '0051246', '0010604', '0031323', '0051173']
['0110165']
['0008150']
['0051049']
['0005575']
['0005575']
['0003674']
['0005575']
['0032989', '0016043']
['0008150']
['0005575']
['0008150']
['0007165']
['0005575']
['0005575']
['0008150']
['0005488', '0003674']
['2001141', '0031326', '0010556', '0060255', '0008150']
['0008150']
['0003674']
['0008150']
['0008150']
['0008150', '0065007']
['0110165']
['0003674']
['0003674']
['0005575']
['0050789']
['0050789', '0048519', '0050794']
['0048869', '0048856']
['0065007']
['000